# **References**

https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-train-lb-0-361

https://www.kaggle.com/rhythmcam/ast-basic-string-expression

https://www.kaggle.com/vexxingbanana/sartorius-coco-dataset-notebook

# **Install Pycocotools**

In [ ]:
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

# **Import Libraries**

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import json
import glob
import pycocotools
from pycocotools import mask
import random
import cv2
import re
import ast

# **Create Coco Json File**

In [ ]:
train_df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')

In [ ]:
train_df

In [ ]:
output_json_dict = {
    "images": [],
    "annotations": [],
    "categories": []
}
category_dict = {"id": 1, "name": "starfish", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)

In [ ]:
annot_id = 0

In [ ]:
def get_boxes(row):
    """Return the bboxes for a given row as a 3D matrix """
    #if len(row['annotations']) == 0:
    #    row['annotations'] = [{'x': -1, 'y': -1, 'width': -1, 'height': -1}]
    return pd.DataFrame(row['annotations'], columns=['x', 'y', 'width', 'height']).astype(float).values

In [ ]:
for f in train_df.itertuples():
    img_path = '../input/tensorflow-great-barrier-reef/train_images/video_' + str(f[1]) + '/' + f[5].split('-')[1] + '.jpg'
    img = cv2.imread(img_path)
    height, width, channels = img.shape

    img_info = {
        "id": f[0],
        "width": width,
        "height": height,
        "file_name": img_path
    }
    output_json_dict["images"].append(img_info)
    if f[6] != '[]':
#         print(train_df.iloc[f[0]]['annotations'])
        bbox_list = ast.literal_eval(f[6])
        for bbox in bbox_list:
#             bbox = ast.literal_eval(bbox)
            if bbox['height'] + bbox['y'] > 720:
                bbox['height'] = 720 - bbox['y']
            annot = {
                "category_id": 1,
                "bbox": [bbox['x'], bbox['y'], bbox['width'], bbox['height']],
                "id": annot_id,
                "image_id": f[0],
                "area": bbox['width'] * bbox['height'],
                "segmentation": [],
                "iscrowd": 0
            }
            output_json_dict["annotations"].append(annot)
            annot_id += 1

In [ ]:
output_json_dict['annotations'][0]

In [ ]:
with open('train_dataset.json', 'w') as f:
    output_json = json.dumps(output_json_dict)
    f.write(output_json)